In [1]:
# checking Python SDKv2
!pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.9.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: msrest, azure-common, tqdm, isodate, marshmallow, pyyaml, typing-extensions, jsonschema, azure-mgmt-core, strictyaml, pyjwt, pydash, opencensus-ext-azure, azure-storage-file-datalake, azure-core, azure-storage-file-share, azure-storage-blob, colorama
Required-by: 


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [4]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


# Train a model and dump it for scoring later
The following part trains a simple model and dump it in a directory as a pickle file in the `model` folder. 

In [ ]:
# Open terminal and move to the src folder and run to save the model
!python train-model-parameters.py --training_data diabetes.csv

## Register the custom model

Batch deployments can only deploy models registered in the workspace. You'll register an custom model, which is stored in the local `model` folder.

https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-model-custom-output?view=azureml-api-2&tabs=python

In [6]:
from azure.ai.ml import MLClient, Input, load_component
from azure.ai.ml.entities import BatchEndpoint, ModelBatchDeployment, ModelBatchDeploymentSettings, PipelineComponentBatchDeployment, Model, AmlCompute, Data, BatchRetrySettings, CodeConfiguration, Environment, Data
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction
from azure.ai.ml.dsl import pipeline
from azure.identity import DefaultAzureCredential

In [13]:
# Registering the model
# Batch Endpoint can only deploy registered models. In this case, we already have a local copy of the model in the repository, 
# so we only need to publish the model to the registry in the workspace. You can skip this step if the model you are trying to deploy is already registered.


model_name = "model"
model_description = "A linear classifier."
model_local_path = "src/model/"

model = ml_client.models.create_or_update(
    Model(
        name=model_name,
        path=model_local_path,
        type=AssetTypes.CUSTOM_MODEL,
        tags={"framework": "scikit-learn", "estimator": "LG"},
    )
)

## Creating a scoring script

We need to create a scoring script that can read the input data provided by the batch deployment and return the scores of the model. We are also going to write directly to the output folder of the job. In summary, the proposed scoring script does as follows:

- Reads the input data as CSV files.
- Runs an MLflow model predict function over the input data.
- Appends the predictions to a pandas.DataFrame along with the input data.
- Writes the data in a file named as the input file, but in parquet format.

### Remarks:

- Notice how the environment variable AZUREML_BI_OUTPUT_PATH is used to get access to the output path of the deployment job.
    - In Azure Machine Learning Studio, the environment variable AZUREML_BI_OUTPUT_PATH is used in the context of batch inferencing. It points to the location where the output results of the batch inferencing should be written.
- The init() function is populating a global variable called output_path that can be used later to know where to write.
- The run method returns a list of the processed files. It is required for the run function to return a list or a pandas.DataFrame object.

In [14]:
%%writefile src/model/score.py

import os
import pickle
import glob
import pandas as pd
from pathlib import Path
from typing import List


def init():
    global model
    global output_path

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    # It is the path to the model folder
    # Please provide your model's folder name if there's one:
    output_path = os.environ["AZUREML_BI_OUTPUT_PATH"]
    model_path = os.environ["AZUREML_MODEL_DIR"]
    model_file = glob.glob(f"{model_path}/*/*.pkl")[-1]

    with open(model_file, "rb") as file:
        model = pickle.load(file)


def run(mini_batch: List[str]):
    for file_path in mini_batch:
        data = pd.read_csv(file_path)
        pred = model.predict(data)

        data["prediction"] = pred

        output_file_name = Path(file_path).stem
        output_file_path = os.path.join(output_path, output_file_name + ".parquet")
        data.to_parquet(output_file_path)

    return mini_batch

Writing src/model/score.py


## Create a batch endpoint

A batch endpoint is an HTTPS endpoint that applications can call to trigger a batch scoring job. A batch endpoint name needs to be unique within an Azure region. You'll use the `datetime` function to generate a unique name based on the current date and time. 

In [15]:
import datetime

endpoint_name = "batch-" + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint_name

'batch-08171837639158'

In [16]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [35]:
# See whether the enviornment is avaialble
envs = ml_client.environments.list()
for my_env in envs:
    print(my_env.name)

batch-inference-ncd-env
DefaultNcdEnv-openmpi4-1-0-ubuntu20-04
farbod-deployment-environment
deployment-environment
DefaultNcdEnv-mlflow-ubuntu20-04-py38-cpu-inference
AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu
AzureML-ACPT-pytorch-1.12-py38-cuda11.6-gpu
AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu
AzureML-ACPT-pytorch-1.11-py38-cuda11.5-gpu
AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu
AzureML-responsibleai-0.21-ubuntu20.04-py38-cpu
AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu
AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu
AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu
AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu
AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu
AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu
AzureML-pytorch-1.8-ubuntu18.04-py37-cuda11-gpu
AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu
AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu
AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda1

In [44]:
deployment = ModelBatchDeployment(
    name="classifier-lm-custom",
    description="a linear model classifier for predicting diabetes",
    endpoint_name=endpoint.name,
    model=model,
    environment= 'AzureML-sklearn-0.24-ubuntu18.04-py37-cpu',
    code_configuration=CodeConfiguration(
        code="./src/model",
        scoring_script="score.py",
    ),
    compute='farbodCluster',
    settings=ModelBatchDeploymentSettings(
        mini_batch_size=2,
        instance_count=2,
        max_concurrency_per_instance=2,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
        logging_level="info",
    ),
)

In [45]:
ml_client.batch_deployments.begin_create_or_update(deployment)

HttpResponseError: (UserError) The Deployment request provided has the following validation errors. Please resolve them and try again.
Errors:
[
  {
    "PropertyName": "ModelConfiguration.EnvironmentId",
    "ErrorMessage": "The value specified for 'Environment Id' is not a valid ARM resource identifier.",
    "AttemptedValue": "azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu",
    "ErrorCode": "ArgumentInvalid"
  }
]

Code: UserError
Message: The Deployment request provided has the following validation errors. Please resolve them and try again.
Errors:
[
  {
    "PropertyName": "ModelConfiguration.EnvironmentId",
    "ErrorMessage": "The value specified for 'Environment Id' is not a valid ARM resource identifier.",
    "AttemptedValue": "azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu",
    "ErrorCode": "ArgumentInvalid"
  }
]

Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "c8892